In [1]:
!pip install --upgrade tables
!pip install eli5
!pip install xgboost
!pip install hyperopt

     |████████████████████████████████| 4.3MB 2.8MB/s 
  Found existing installation: tables 3.4.4
    Uninstalling tables-3.4.4:
      Successfully uninstalled tables-3.4.4
     |████████████████████████████████| 112kB 2.8MB/s 


In [2]:
import pandas as pd
import numpy as np

import xgboost as xgb
from hyperopt import hp, fmin, tpe, STATUS_OK

from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import cross_val_score, KFold

import eli5
from eli5.sklearn import PermutationImportance

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)
Using TensorFlow backend.


In [3]:
cd '/content/drive/My Drive/Colab Notebooks/dw_matrix/data'

/content/drive/My Drive/Colab Notebooks/dw_matrix/data


In [0]:
df = pd.read_hdf('car_data.h5')

In [0]:
df.rename(columns = {'param_skrzynia-biegów':'param_skrzynia_biegów'}, inplace = True)

In [0]:
df['param_typ__cat'] = df['param_typ'].factorize()[0]

In [0]:
df['param_przebieg_num'] = df['param_przebieg'].map( lambda x: -1 if str(x) == 'None' else int(str(x).split('km')[0].replace(' ', '')) )

In [0]:
def run_model(model, feats):
  X = df[feats].values
  y = df['price_value'].values

  scores = cross_val_score(model, X, y, cv=3, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [0]:
feats = ['param_napęd__cat','param_rok-produkcji','param_stan__cat','param_skrzynia-biegów__cat','param_faktura-vat__cat','param_moc','param_marka-pojazdu__cat',
         'param_typ__cat','feature_kamera-cofania__cat','param_pojemność-skokowa','seller_name__cat','param_kod-silnika__cat','param_model_pojazdu__cat',
         'feature_wspomaganie-kierownicy__cat','param_wersja__cat','feature_czujniki-parkowania-przednie__cat','feature_asystent-pasa-ruchu__cat','feature_system-start-stop__cat',
         'feature_regulowane-zawieszenie__cat','feature_światła-led__cat', 'param_przebieg_num']

In [0]:
xgb_params = {'max_depth': 5,
              'n_estimators': 50,
              'learning_rate': 0.1,
              'seed': 2020
              }

In [0]:
model = xgb.XGBRegressor(**xgb_params)

In [12]:
run_model(model, feats)

[15:59:47] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:59:52] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:59:56] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-9312.544136473989, 69.39471346674082)

In [0]:
!git config --global user.email "m.zajac1988@gmail.com"
!git config --global user.name "Mateusz"

In [14]:
cd '/content/drive/My Drive/Colab Notebooks/dw_matrix'

/content/drive/My Drive/Colab Notebooks/dw_matrix


In [0]:
!git add matrix_two/day5_hyperopt.ipynb

In [16]:
!git commit -m 'Initial setup'

[master 3769f4d] Initial setup
 1 file changed, 1 insertion(+)
 create mode 100644 matrix_two/day5_hyperopt.ipynb


In [17]:
!git push -u origin master

Counting objects: 4, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (4/4), done.
Writing objects: 100% (4/4), 3.54 KiB | 1.18 MiB/s, done.
Total 4 (delta 2), reused 0 (delta 0)
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To https://github.com/mattzajac/dw_matrix.git
   1cf96f8..3769f4d  master -> master
Branch 'master' set up to track remote branch 'master' from 'origin'.


In [0]:
def obj_func(params):
  print('Training with params: ')
  print(params)

  mean_mae, score_std = run_model(xgb.XGBRegressor(**params), feats)
  return {'loss': np.abs(mean_mae), 'status': STATUS_OK}

In [0]:
# space
space = {
    'learning_rate': hp.choice('learning_rate', np.arange(0.05, 0.31, 0.05)),
    'max_depth':  hp.choice('max_depth', np.arange(5, 20, 1, dtype=int)),
    'subsample': hp.quniform('subsample', 0.5, 1, 0.05),
    'colsample_bytree': hp.quniform('colsample_bytree', 0.5, 1, 0.05),
    'objective': 'reg:squarederror',
    'n_estimators': 100,
    'seed': 2020
    }

In [25]:
#run
best = fmin(obj_func, space, algo=tpe.suggest, max_evals=25)
best

Training with params: 
{'colsample_bytree': 0.65, 'learning_rate': 0.25, 'max_depth': 19, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 2020, 'subsample': 0.55}
Training with params: 
{'colsample_bytree': 0.55, 'learning_rate': 0.2, 'max_depth': 17, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 2020, 'subsample': 0.9}
Training with params: 
{'colsample_bytree': 0.8500000000000001, 'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 2020, 'subsample': 0.8500000000000001}
Training with params: 
{'colsample_bytree': 0.75, 'learning_rate': 0.3, 'max_depth': 11, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 2020, 'subsample': 0.65}
Training with params: 
{'colsample_bytree': 0.8, 'learning_rate': 0.2, 'max_depth': 7, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 2020, 'subsample': 0.7000000000000001}
Training with params: 
{'colsample_bytree': 1.0, 'learning_rate': 0.1, 'max_depth

{'colsample_bytree': 0.8,
 'learning_rate': 1,
 'max_depth': 10,
 'subsample': 1.0}